# Using RandomForest to Predict Survival Rate of Titanic

# (1): Let's start by visualizing the data from train.csv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_csv('/kaggle/input/titanic/train.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

Check the survival rate

In [ ]:
survived_rate = df.iloc[1:]['Survived'].mean()
plt.bar(['Survived', 'Not survived'], [survived_rate, 1-survived_rate])
plt.ylim(0, 1)
plt.title('Survived vs Not Survived')
plt.grid(True)
plt.text(0, survived_rate, f'{survived_rate:.3f}', ha='center', va='bottom')
plt.text(1, 1-survived_rate, f'{1-survived_rate:.3f}', ha='center', va='bottom')
plt.show()

Check the Pclass

In [ ]:
df["Pclass"].describe()

In [ ]:
df["Pclass"].value_counts()

We then check the survival rates of different categories of Pclass

In [ ]:
df.groupby("Pclass")["Survived"].mean()

Check the mean survival rates of different bins of Age

In [ ]:
survival_age = df.groupby(pd.cut(df['Age'], bins=range(0, 100, 5)), observed=True)['Survived'].mean()
survival_age

In [ ]:
plt.bar(survival_age.index.astype(str), survival_age)
plt.xticks(rotation=45)
plt.show()

The frequency of both sex, and the survival rate

In [ ]:
df["Sex"].value_counts(normalize=True)

In [ ]:
df.groupby("Sex")["Survived"].mean()

Survival rate for people with how many siblings/spouses

In [ ]:
df.groupby("SibSp")["Survived"].mean()

The survival rate of people with how many parents/children

In [ ]:
df.groupby("Parch")["Survived"].mean()

Distribution of fares

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['Fare'], bins=50, edgecolor='black')
plt.title('Distribution of Fare Prices')
plt.xlabel('Fare')
plt.ylabel('Count')
plt.show()

Distribution of fares and survival rates

In [ ]:
fare_bins = pd.cut(df['Fare'], bins=range(0, int(df['Fare'].max())+50, 50))
survival_by_fare = df.groupby(fare_bins, observed=True)['Survived'].mean()
plt.figure(figsize=(12, 6))
plt.bar(range(len(survival_by_fare)), survival_by_fare)
plt.title('Survival Rate by Fare Price')
plt.xlabel('Fare Range')
plt.ylabel('Survival Rate')
plt.xticks(range(len(survival_by_fare)), survival_by_fare.index.astype(str), rotation=45)
plt.show()

Port of embarkation frequency and survival rate distribution

In [ ]:
df["Embarked"].value_counts(normalize=True)

In [ ]:
df.groupby("Embarked")["Survived"].mean()

According to our data visualization, the survival rate of women is significantly higher than that of men, the survival rate of first class is higher than that of first class than that of second class, the survival rate of children is higher than that of other ages, the survival rate of people with 1-2 siblings/spouses is significantly higher than that of others, the survival rate of people with 1-2 parents/children is significantly higher than that of others, and the survival rate of people with tickets above 500 is the highest. Those with fares below 50 have the lowest survival rates, and the port of embarkation in Cherbourg has a significantly higher survival rate than the others.

（2）baseline：logistic regression with basic features-accuracy 0.73

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
features = ['Pclass', 'Sex', 'Age', 'Fare', 'SibSp', 'Parch']
X = train_data[features].copy()
y = train_data['Survived']
X_test = test_data[features].copy()
X['Age'] = X['Age'].fillna(X['Age'].median())
X_test['Age'] = X_test['Age'].fillna(X_test['Age'].median())
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].median())
X['Sex'] = X['Sex'].map({'female': 1, 'male': 0})
X_test['Sex'] = X_test['Sex'].map({'female': 1, 'male': 0})
model = LogisticRegression(random_state=42)
model.fit(X, y)
predictions = model.predict(X_test)
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': predictions
})
submission.to_csv('submission.csv', index=False)
print("Feature Importance:")
for feature, importance in zip(features, abs(model.coef_[0])):
    print(f"{feature}: {importance:.4f}")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
val_model = LogisticRegression(random_state=42)
val_model.fit(X_train, y_train)
val_score = val_model.score(X_val, y_val)
print(f"\nAccuracy: {val_score:.4f}")

baseline 2: RandomForest with basic features and feature engineering-accuracy 0.74

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
features = ['Pclass', 'Sex', 'Age', 'Fare', 'SibSp', 'Parch']
X = train_data[features].copy()
y = train_data['Survived']
X_test = test_data[features].copy()
X['Age'] = X['Age'].fillna(X['Age'].median())
X_test['Age'] = X_test['Age'].fillna(X_test['Age'].median())
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].median())
X['Sex'] = X['Sex'].map({'female': 1, 'male': 0})
X_test['Sex'] = X_test['Sex'].map({'female': 1, 'male': 0})
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)
predictions = model.predict(X_test)
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': predictions
})
submission.to_csv('submission.csv', index=False)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
val_model = RandomForestClassifier(n_estimators=100, random_state=42)
val_model.fit(X_train, y_train)
val_score = val_model.score(X_val, y_val)
print(f"\nAccuracy: {val_score:.4f}")

# (2)：Within baseline, We started doing more detailed data preprocessing

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
df = train_data
df1 = test_data

Check the null values

In [ ]:
df.isnull().sum()

In [ ]:
df1.isnull().sum()

Now check the updated status of all features.

In [ ]:
df.head()

In [ ]:
df.describe()

# (3) Feature Engineering

We try to handel Null values.
fill null values with :

median for numerical feaures
mode for categorical features
Or using linear regression

Next we try to extract all titles in the Name column and check the frequencies.

In [ ]:
# Extract title from the Name column
train_titles = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test_titles = df1['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
all_titles = pd.concat([train_titles, test_titles]).unique()
# Sort the titles and convert them to a list
titles_list = sorted(list(all_titles))
print("Titles：")
print(titles_list)
# Check the frequency of title

print("\nThe frequency of titles in train.csv：")
print(train_titles.value_counts())
print("\nThe frequency of title in test.csv：")
print(test_titles.value_counts())

In [ ]:
from sklearn.linear_model import LinearRegression
# First we perform the basic feature transformation
for df2 in [df, df1]:
    # Gender coding and basic characteristics processing
    df2['Sex'] = df2['Sex'].map({'female': 0, 'male': 1})
    df2['Fare'] = df2['Fare'].fillna(df2['Fare'].median())
    df2['Embarked'] = df2['Embarked'].fillna(df2['Embarked'].mode()[0])
    df2['Embarked'] = df2['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
    # Title extraction and mapping
    df2['Title'] = df2['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    title_map = {
        'Mr': 1, 'Don': 1,                    # Civilian males
        'Miss': 2, 'Mlle': 2,                 # Unmarried women
        'Mrs': 3, 'Ms': 3, 'Mme': 3,         # Married women
        'Master': 4,                          # Young
        'Countess': 5, 'Lady': 5, 'Dona': 5, # Noble wommen
        'Jonkheer': 6, 'Sir': 6,             # Noble males
        'Dr': 7,                             # Doctors
        'Rev': 8,                            # Priests
        'Col': 9, 'Major': 9, 'Capt': 9      # Specials
    }
    df2['Title'] = df2['Title'].map(title_map)

# Age prediction for the training set and the test set were processed separately by linear regression.
age_features_train = ['Pclass', 'Sex', 'Fare', 'Title', 'Survived', 'SibSp', 'Parch']
known_age_train = df[df['Age'].notna()]
unknown_age_train = df[df['Age'].isna()]
lr_model_train = LinearRegression(positive=True)
lr_model_train.fit(known_age_train[age_features_train], known_age_train['Age'])
predicted_ages_train = lr_model_train.predict(unknown_age_train[age_features_train])
df.loc[df['Age'].isna(), 'Age'] = predicted_ages_train

age_features_test = ['Pclass', 'Sex', 'Fare', 'Title', 'SibSp', 'Parch']
known_age_train = df[df['Age'].notna()] 
lr_model_test = LinearRegression(positive=True)
lr_model_test.fit(known_age_train[age_features_test], known_age_train['Age'])
unknown_age_test = df1[df1['Age'].isna()]
predicted_ages_test = lr_model_test.predict(unknown_age_test[age_features_test])
df1.loc[df1['Age'].isna(), 'Age'] = predicted_ages_test

for df2 in [df, df1]:
    # Deck mapping
    df2['Deck'] = df2['Cabin'].fillna('U').str[0]
    deck_map = {
        'A': 1, 'B': 2, 'C': 3, 'D': 4,
        'E': 5, 'F': 6, 'G': 7, 'T': 8, 'U': 0
    }
    df2['Deck'] = df2['Deck'].map(deck_map)    
    # Using fares to predict Deck
    known_deck = df2[df2['Deck'] != 0]
    lr_deck = LinearRegression()
    lr_deck.fit(known_deck[['Fare', 'Title']], known_deck['Deck'])
    unknown_deck = df2[df2['Deck'] == 0]
    predicted_deck = lr_deck.predict(unknown_deck[['Fare', 'Title']])
    df2.loc[df2['Deck'] == 0, 'Deck'] = np.round(predicted_deck)
    
    # Create composite features
    df2['FamilySize'] = df2['SibSp'] + df2['Parch'] + 1
    df2['Agebin'] = pd.qcut(df2['Age'], q=5, labels=False)
    df2['Farebin'] = pd.qcut(df2['Fare'], q=5, labels=False)
    df2['IsAlone'] = (df2['FamilySize'] == 1).astype(int)
    df2['PclassSex'] = df2['Pclass'].astype(str) + df2['Sex'].astype(str)
    df2['Fare/FamilySize'] = df2['Fare']/df2['FamilySize']
    df2['Age_Class'] = df2['Age'] * df2['Pclass']
    df2['FreeFare'] = df2['Fare'] == 0
    # PclassSex mapping
    pclass_sex_map = {
        '10': 1, '11': 2,  
        '20': 3, '21': 4, 
        '30': 5, '31': 6  
    }
    df2['PclassSex'] = df2['PclassSex'].map(pclass_sex_map)

# (4): Model prediction and evaluation

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
# Choose the features
features = ['PclassSex', 'Agebin', 'Farebin', 'Embarked', 'FamilySize', 'Deck', 'Title', 'Fare/FamilySize', 'IsAlone', 'FreeFare']

# Since Agebin and Farebin are categorical variables, unique thermal encoding is required
X = pd.get_dummies(df[features])
y = df['Survived']
X_test = pd.get_dummies(df1[features], columns=['Agebin', 'Farebin'])
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# Split train data and test data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Random Forest
rf_model = RandomForestClassifier(n_estimators=500, random_state=42, min_samples_leaf=5, min_samples_split=5, max_depth=3, max_features='log2')
rf_model.fit(X_train, y_train)
rf_score = rf_model.score(X_val, y_val)
print(f"Random Forest Score: {rf_score:.4f}")

# Cross validation
scores = cross_val_score(rf_model, X, y, cv=5)
print(f"\nRandom Forest Cross Validation Scores:")
print(f"Mean: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

# Prediction
rf_predictions = rf_model.predict(X_test) 
rf_submission = pd.DataFrame({
    'PassengerId': df1['PassengerId'],
    'Survived': rf_predictions
})
rf_submission.to_csv('rf_titanic.csv', index=False)

In [ ]:
# Output detailed categorical reports
y_val_pred = rf_model.predict(X_val)
print("\nReport:")
print(classification_report(y_val, y_val_pred))

## We found that the recall rate of "1" is low(70%) compared to "0"(90%). Thus, we try to do sampling with SMOTETomek and clean up the sample to remove some of the most likely classes of noise

In [ ]:
from imblearn.combine import SMOTETomek
# Using SMOTETomek
smote = SMOTETomek(sampling_strategy=0.6, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# View the category distribution after resampling
print("\nCategory distribution after resampling:")
print(pd.Series(y_train_resampled).value_counts())

# Random Forest
rf_model = RandomForestClassifier(
    n_estimators=500, 
    random_state=42, 
    min_samples_leaf=5, 
    min_samples_split=5, 
    max_depth=3, 
    max_features='log2'
)
rf_model.fit(X_train_resampled, y_train_resampled)

rf_score = rf_model.score(X_val, y_val)
print(f"\nRandom Forest Score on Validation Set: {rf_score:.4f}")

# Output detailed categorical reports
y_val_pred = rf_model.predict(X_val)
print("\nCategorial Report:")
print(classification_report(y_val, y_val_pred))

# Cross validation
scores = cross_val_score(rf_model, X, y, cv=5)
print(f"\nRandom Forest Cross Validation Scores:")
print(f"Mean: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

# Check the feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
})
print("\nFeature Importance:")
print(feature_importance.sort_values('importance', ascending=False))

# Final prediction
rf_predictions = rf_model.predict(X_test)
rf_submission = pd.DataFrame({
    'PassengerId': df1['PassengerId'],
    'Survived': rf_predictions
})
rf_submission.to_csv('rf_prediction_with_SMOTE.csv', index=False)
rf_submission.to_csv('submission.csv', index=False)

## Finally, the accuracy is 0.7990